# Klasifikasi Berita Liputan6

In [15]:
# import library

import pandas as pd
import numpy as np

In [70]:
# load data

df = pd.read_csv("raw.csv")
df.head()

,Judul
0,Kondisi Kejiwaan Yusuf Mansur Disorot Akibat K...
1,Nagita Slavina dan RANS Trending Usai Disebut ...
2,Berita Indonesia vs Australia Hari Ini - Kabar...
3,Alas Roban Jawa Tengah Penampakan Bikin Sopir ...


In [ ]:
# periksa tipe datanya utk masing2 varibael/kolom/fitur dengan .dtypes

print(df.dtypes)

In [ ]:
# melihat statistik dasar 

df.describe()

## Membuang (drop) Kolom
---

* Membuang Kolom pada DataFrame
* Sering ditemukan beberapa kategori data tidak terlalu berguna di dataset, misal untuk menganalisis IPK mahasiswa , data nama orangtua, alamat adalah data tidak penting
* Pandas menyediakan fungsi untuk membuang (drop) kolom-kolom yang tidak diinginkan dengan fungsi <code style="background:#E5E7E9 ;color:black">drop()</code>.

In [ ]:
# melihat pada lima entri pertama 

df.head()

Dapat dilihat bahwa beberapa kolom memberikan informasi tambahan yang akan membantu perpustakaan tetapi tidak terlalu deskriptif tentang buku itu sendiri, diantaranya kolom: *Edition Statement, Corporate Author, Corporate Contributors, Former owner, Engraver, Issuance type and Shelfmarks.*

In [ ]:
# memilih kolom-kolom yang akan di drop 

to_drop = ['Edition Statement', 
           'Corporate Author', 
           'Corporate Contributors', 
           'Former owner', 
           'Engraver',
           'Contributors',
           'Issuance type',
           'Shelfmarks']

Kita definisikan daftar (list) nama dari semua kolom yang ingin kita drop. Kemudian jalankan perintah fungsi <code style="background:#E5E7E9 ;color:black">drop()</code>,  dengan melewatkan parameter inplace bernilai True dan parameter axis bernilai 1

In [ ]:
# melakukan drop pada data yang dipilih

df.drop(to_drop, inplace=True, axis=1)

In [ ]:
# inspeksi ulang DataFrame, kolom yang tidak diinginkan sudah dibuang

df.head()

#### Alternatif untuk membuang kolom, dengan meneruskannya langsung ke parameter columns daripada memisahkan label-label yang mau dibuang.

In [ ]:
#df.drop(colums=to_drop, inplace=True)

### Sintak di atas ini lebih intuitif dan mudah dibaca, bukan?.

In [ ]:
# cek ulang data

df.head()

## Mengubah Indeks di DataFrame
---


* Index dalam Pandas memperluas fungsionalitas array NumPy untuk memungkinkan pemotongan (slicing) dan pelabelan yang lebih fleksibel. Dalam banyak kasus, akan sangat membantu jika menggunakan field pengenal data yang bernilai unik sebagai indeksnya.
* Sebagai contoh, dengan dataset di slide sebelumnya, praktiknya saat pustakawan mencari record, biasanya akan memasukan identifier unik suatu buku:

In [ ]:
# cek nilai unik identifier

df['Identifier'].is_unique

In [ ]:
# ganti indeks yang ada pada kolom menggunakan set_index

df = df.set_index('Identifier')
df.head()

Kita dapat mengakses setiap records dengan cara yang mudah dengan <code style="background:#E5E7E9 ;color:black">loc[]</code>. Cara ini digunakan untuk *label-based indexing*, yaitu memberi label suatu baris atau kolom tanpa memperhatikan posisi/lokasinya.

In [ ]:
# akses record ke 206

df.loc[206]

# .loc[] --> seleksi dengan menggunakan label/bilangan bulat  

#### Dengan kata lain, 206 adalah label pertama dari indeks. 

* Untuk mengakses berdasarkan posisinya, gunakan <code style="background:#E5E7E9 ;color:black">df.iloc[]</code>

* Pada slide sebelumnya, Indeks yang digunakan adalah RangeIndex: integer mulai dari 0, analog dengan range di Python. Dengan meneruskan nama kolom ke set_index, maka indeks telah diubah ke nilai dalam Identifier.

* Diperhatikan pada langkah sebelumnya bahwa telah dilakukan penetapan kembali variabel ke objek yang dikembalikan oleh metode dengan df = df.set_index(...). Ini karena, secara default, metode mengembalikan salinan objek yang dimodifikasi dan tidak membuat perubahan secara langsung ke objek. Hal ini dapat dihindari dengan mengatur parameter inplace:

In [ ]:
#df.set_index('Identifier', inplace=True)

## Merapihkan Fields dalam Data 
---

* Slide sebelumnya telah dibuang beberapa kolom tidak penting dan diubah indeks pada DataFrame hingga menjadi lebih masuk akal. 
* Selanjutnya, akan dibersihkan kolom tertentu dan mengubah menjadi bentuk/format yang seragam hingga dataset lebih mudah dipahami dan memastikan konsistensi. Dalam slide berikutnya akan dibersihkan Date of Publication dan  Place of Publication.
* Dalam inspeksi, semua tipe data saat ini adalah objek dtype yang analog dengan str di native Python
* Cara ini dilakukan sebagai rangkuman saat setiap field tidak dapat dirapihkan sebagai data numerik atau data kategorik dan data yang digunakan cukup “kotor” atau “berantakan”. 

In [ ]:
# menghitung frekuensi nilai objek

df.dtypes.value_counts()

##### Satu kolom yang masuk akal untuk menerapkan nilai numerik adalah tanggal publikasi sehingga kita dapat melakukan perhitungan di awal:

In [ ]:
# seleksi data dari 1905 dengan output hanya pada field tanggal publikasi

df.loc[1905:, 'Date of Publication'].head(10)

Buku tertentu hanya memiliki satu tanggal publikasi. Oleh karena itu perlu dilakukan hal berikut:

    1. Hilangkan tanggal lain dalam kurung siku, 1879[1878]
    2. Konversi rentang tanggal ke “start date”, 1860-63; 1839, 38-54
    3. Hilangkan tanggal yang tidak jelas dan gantikan dengan NaN NumPy, [1879?] -> NaN
    4. Konversi string nan ke nilai NaN  NumPy

In [ ]:
# menggunakan ekspresi reguler (Regex) tunggal untuk mengekstrak tahun publikasi

regex = r'^(\d{4})'

Catatan :

- Perintah <code style="background:#E5E7E9 ;color:black">\d</code> mewakili sebarang digit 
- <code style="background:#E5E7E9 ;color:black">{4}</code> mengulangi aturan (rule) sebanyak empat kali. 
- Kararakter <code style="background:#E5E7E9 ;color:black">^</code> sesuai dengan awal string, dan tanda dalam kurung 
- <code style="background:#E5E7E9 ;color:black">()</code> menunjukkan capturing group yang memberikan sinyal ke Pandas bahwa akan dilakukan ekstraksi bagian Regex tersebut. 

In [ ]:
# jalankan regex di dataset

extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.head()

# expand=False: --> Jika Benar, kembalikan  DataFrame dengan satu kolom per grup tangkapan.  
# Jika Salah, kembalikan Seri/Indeks jika ada satu  grup tangkapan atau DataFrame jika ada beberapa  grup tangkapan.

Secara teknis, kolom tsb masih memiliki object dtype, namun dengan mudah kita dapatkan versi numeriknya dengan perintah <code style="background:#E5E7E9 ;color:black">pd.to_numeric</code>


In [ ]:
# mendapatkan versi numeriknya dengan perintah pd.to_numeric

df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

Ini menghasilkan sekitar 1/10 nilai yang hilang, cost yang cukup kecil dampaknya untuk saat ini karena dapat melakukan perhitungan pada nilai valid yang tersisa:


In [ ]:
# perhitungan pada nilai valid yang tersisa

df['Date of Publication'].isnull().sum() / len(df)

## Membersihkan Kolom dengan Kombinasi metode str dengan NumPy 
---
* Slide sebelumnya dibahas penggunaan df['Date of Publication'].str. Atribut ini adalah cara akses cepat operasi string di Pandas yang menyerupai operasi pada native Python atau mengkompilasi regex seperti <code style="background:#E5E7E9 ;color:black">.split()</code>, <code style="background:#E5E7E9 ;color:black">.replace()</code>, dan <code style="background:#E5E7E9 ;color:black">.capitalize()</code>.
* Untuk membersihkan field Place of Publication, kombinasikan metode str di Panda dengan fungsi np.where di NumPy yang mirip dengan bentuk vektor dari makro IF() di Excell, dengan sintak berikut:

In [ ]:
#np.where(condition, then, else)

* condition mirip dengan objek array atau Boolean .then adalah nilai yang digunakan jika condition mengevaluasi menjadi True, dan else untuk mengevaluasi nilai selainnya.

* <code style="background:#E5E7E9 ;color:black">.capitalize().where</code> membawa tiap elemen dalam objek digunakan untuk condition dan memeriksa elemen tertentu menjadi True dalam konteks kondisi dan mengembalikan ndarray terdiri dari then atau else, tergantung pada prakteknya.


* Dapat juga dituliskan dalam berkalang (nested) menjadi pernyataan If-Then, memungkinkan menghitung nilai berbasiskan kondisi berganda:


In [ ]:
#np.where(condition1, x1, 
        #np.where(condition2, x2, 
            #np.where(condition3, x3, ...)))

* Kemudian, dapat digunakan dua fungsi tsb untuk membersihkan field Place of Publication karena kolom tsb memiliki objek string. Berikut adalah isi dari kolom:


In [ ]:
# isi kolom 'Place of Publication'

df['Place of Publication'].head(10)

* Dilihat pada hasil di atas, field  place of publication masih ada informasi yang tidak penting. Jika dilihat lebih teliti, kasus ini untuk beberapa baris yang place of publication -nya di “London” dan “Oxford”


In [ ]:
df.loc[4157862]

In [ ]:
df.loc[4159587]

* Pada dua entri di atas, dua buku  diterbitkan di tempat yang sama (newcastle upon tyne) namun salah satunya memilik tanda hubung (-) 
* Untuk membersihkan kolom ini dalam sekali jalan, gunakan <code style="background:#E5E7E9 ;color:black">str.contains()</code> untuk mendapatkan Boolean mask.


In [ ]:
# mendapatkan boolean mask

pub = df['Place of Publication']
london = pub.str.contains('London')
london[:5]

In [ ]:
oxford = pub.str.contains('Oxford')

* Kombinasikan dengan np.where:

In [ ]:
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                               pub.str.replace('-', ' ')))
df['Place of Publication'].head()

* Di sini, fungsi np.where berbentuk struktur nested, dimana condition berbentuk Series dari Boolean dengan str.contains(). Metode contains() bekerja mirip dengan keyword in yang digunakan untuk mencari kejadian suatu entitas  dalam kondisi pengulangan iterasi (atau substring dalam suatu string) 

* Pergantian tanda hubung (hypen) dengan spasi dengan str.replace() dan re-assign ke kolom dalam DataFrame.
* Walau pada kenyataan masih banyak dataset ini (kolom dan baris) yang “kotor” , namun dalam contoh di sini hanya dibahas pada dua kolom

* Coba periksa kembali untuk lima entri pertama, hasilnya akan lebih rapih dan “bersih” dibandingkan dataset awal sebelum dilakukan cleaning data.




In [ ]:
# menampilkan data

df.head()

## Membersihkan Seluruh Dataset dengan Fungsi applymap
---
* Pada situasi tertentu, data berantakan alias “kotor” tidak hanya berlaku di sebagian kolom atau baris (record) tapi menyebar ke banyak bagian dataset. 
* Cara berikut dapat diterapkan untuk semua cell atau elemen di DataFrame (dataset).
* Metode .applymap() dapat diterapkan, dimana similar dengan fungsi built-in yaitu fungsi map().  


* Terapkan fungsi applymap() pada file “university_towns.txt”:



![image.png](attachment:image.png)

                                             tampilan Command Promt pada windows

![image-2.png](attachment:image-2.png)
                                                                
                                                  tampilan terminal Linux

* Dapat dilihat di atas, bahwa nama negara bagian(state) diikuti dengan kota asal universitas: StateA TownA1 TownA2 StateB TownB1 TownB2.... 
dan memiliki substring ”[edit]” 


* Kita dapat memanfaatkan pola ini dengan membuat list of (state, city) tuples dan wrapping daftar (list) dalam DataFrame. 



In [ ]:
university_towns = []
with open("university_towns.txt") as file:
     for line in file:
         if '[edit]' in line:
             # Remember this `state` until the next is found
             state = line
         else:
             # Otherwise, we have a city; keep `state` as last-seen
             university_towns.append((state, line))

university_towns[:5]

* Kita dapat membungkus (wrap) daftar ini dalam DataFrame dan mengatur kolom sebagai “State” and “RegionName”.
* Pandas akan mengambil setiap elemen dalam daftar dan mengatur "State" ke nilai kiri dan “RegionName” ke nilai kanan.
* Hasilnya adalah DataFrame sbb:

In [ ]:
towns_df = pd.DataFrame(university_towns,
                        columns=['State', 'RegionName'])

towns_df.head()

* Pandas, mempermudah dalam pembersihan string dengan hanya membutuhkan nama state  dan nama town dan dapat membuang lainnya. Selain dapat kembali menggunakan metode  .str() di Pandas, dapat juga menggunakan applymap() untuk memetakan setiap elemen di DataFrame
* Perhatikan kasus sederhana pada contoh DataFrame berikut:  


In [ ]:
contoh_data = {0 : ['mock','python','real','numpy'],
               1 : ['Dataset','pandas','python','clean']}

contoh_dataframe = pd.DataFrame(contoh_data)

contoh_dataframe.head()

* Pada contoh di atas, setiap sel (“Mock”, “Dataset”, “Python”, “Real”, dll) adalah elemen. Oleh karena itu perintah applymap() akan menerapkan fungsi ke setiap elemen secara independen. Mari kita definisikan fungsi tsb:  


In [ ]:
def get_citystate(item):
     if ' (' in item:
         return item[:item.find(' (')]
     elif '[' in item:
         return item[:item.find('[')]
     else:
         return item

* applymap() di Pandas hanya butuh satu parameter , yaitu fungsi yang diterapkan ke setiap elemen: 


In [ ]:
towns_df =  towns_df.applymap(get_citystate)

* Pertama, definisikan fungsi Python yang mengambil setiap elemen  dari DataFrame sebagai parameternya. Di dalam fungsi, pengecekan dilakukan utk menentukan apakah ada elemen atau tidak!
* Tergantung pada pengecekan, nilai dikembalikan berdasarkan fungsi. 
* Lalu, fungsi applymap() dipanggil pada objek yg ada. Sehingga kita dapatkan DataFrame yang relatif lebigh rapih



In [ ]:
towns_df.head()

## Mengganti Nama Kolom 
---

* Seringkali dalam dataset yang dimiliki ada nama kolom yang suli utk dipahami atau informasi yang tidak penting dalam beberapa baris awal/akhir, misal definisi istilah atau footnotes.
* Oleh karena itu dapat dilakukan penggantian nama dan melewatkan beberapa baris sehingga bisa dilakukan analisis informasi dari baris yang benar atau dapat dipahami. 
* Kita akan lakukan untuk lima baris awal dataset “olympic.csv”:

![image.png](attachment:image.png)

                                                Tampilan Command Prompt Windows

![image.png](attachment:image.png)

                                                    Tampilan Terminal Linux

* Kemudian, baca dalam DataFrame di Pandas:


In [ ]:
olympics_df = pd.read_csv("olympics.csv")
olympics_df.head()

* Hasilnya berantakan! Kolom adalah bentuk string integer indeks 0. Baris yang harusnya sebegai header pada olympics_df.iloc[0]. Hal ini terjadi karena file CSV mulai dengan 0, 1, 2, ...., 15. 
* Dan, jika kita ke sumber dataset ini, akan terlihat NaN yang ada harusnya berisikan “Country” dan “Summer” maksudnya adala”Summer Games” dan “01!” harusnya adalah “Gold” , dll. 


* Oleh karena itu, hal berikut yang perlu dilakukan:
    1. Melewatkan (skip) satu baris dan atur header sebagai baris pertama (indeks-0)
    2. Mengganti Nama Kolom
 
 
 

* Melewatkan baris dan atur header dapat dilakukan pada saat membaca file CSV dengan mempassing beberapa parameter ke fungsi read_csv().
* Fungsi read_csv() memerlukan banyak parameter opsional, namun utk kasus ini hanya diperlukan satu (header) yang dihilangkan pada baris ke-0, dengan hasil sbb:


* Hasil fungsi read_csv() dan menghilangkan satu baris (header):

In [ ]:
olympics_df = pd.read_csv("olympics.csv", header=1)
olympics_df.head()

* Sekarang, yang tampak di atas adalah sekumpulan baris yang benar sebagai header dan semua baris yang tidak dibutuhkan telah dihilangkan. 
* Pandas telah merubah nama kolom yang mengandung nama “countries” dari NaN  menjadi Unnamed:0


* Utk mengganti nama kolom, digunakan metode rename() DataFrame yg memungkinkan memberi label pada axis berdasarkan pemetaan (dalam kasus ini yaitu dict)
* Mulai dengan mendefinisikan suatu kamus yang memetakan nama kolom saat ini sebagi kunci ke yang lebih dapat digunakan” 

In [ ]:
new_names =  {'Unnamed: 0': 'Country',
               '? Summer': 'Summer Olympics',
               '01 !': 'Gold',
               '02 !': 'Silver',
               '03 !': 'Bronze',
               '? Winter': 'Winter Olympics',
               '01 !.1': 'Gold.1',
               '02 !.1': 'Silver.1',
               '03 !.1': 'Bronze.1',
               '? Games': '# Games',
               '01 !.2': 'Gold.2',
               '02 !.2': 'Silver.2',
               '03 !.2': 'Bronze.2'}

* Kemudian, panggil fungsi rename() pada objek dimaksud:


In [ ]:
olympics_df.rename(columns=new_names, inplace=True)

* Atur inplace menjadi True,
     dengan hasil sbb:


In [ ]:
olympics_df.head()